1) Design model(input, output size, forward pass)
2) Construct loss and optimizer
3) Training loop
- forward pass : compute prediction and loss
- backward pass : gradients
- update weights


In [16]:
import torch
import torch.nn as nn 
import numpy as np 
from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [17]:
# 0) Prepare Data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=21)

#scale features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(-1,1)
y_test = y_test.view(-1,1)

In [18]:
# 1) Model
class LRModel(nn.Module):
    def __init__(self,n_input_features):
        super().__init__()
        self.linear = nn.Linear(n_input_features,1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LRModel(n_features)


In [19]:
# 2) Loss and Optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)


In [20]:
# 3) Training Loop
n_epochs = 100

for epoch in range(n_epochs):
    #forward pass
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    #backward pass
    loss.backward()
    #updates
    optimizer.step()

    # flush gradients

    optimizer.zero_grad()

    if (epoch+1)%10==0:
        print(f'epcoch : {epoch+1}, loss = {loss.item():.4f}')

epcoch : 10, loss = 0.6786
epcoch : 20, loss = 0.5483
epcoch : 30, loss = 0.4689
epcoch : 40, loss = 0.4154
epcoch : 50, loss = 0.3767
epcoch : 60, loss = 0.3472
epcoch : 70, loss = 0.3239
epcoch : 80, loss = 0.3048
epcoch : 90, loss = 0.2890
epcoch : 100, loss = 0.2755


In [21]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted = y_predicted.round()
    acc = y_predicted.eq(y_test).sum()/ float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9561
